# Italy

In [565]:
import io
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup

## Fetch results

In [58]:
base_url = 'https://elezionistorico.interno.gov.it'

In [63]:
def get_panel(url=None):
    if url:
        url = f'{base_url}{url}'
    else:
        url = f'{base_url}/index.php?tpel=E&dtel=25/05/2014&tpa=I&tpe=A&lev0=0&levsut0=0&es0=S&ms=S'
    user_agent = ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36')
    r = requests.get(url, headers={'User-Agent': user_agent})
    html = BeautifulSoup(r.text)
    panel = html.select('.panel-default')[-1]
    c = panel.select('.sezione li > a')
    return [(x.attrs['href'], x.attrs['title']) for x in c]

In [67]:
def get_turnout(url):
    user_agent = ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36')
    r = requests.get(f'{base_url}{url}', headers={'User-Agent': user_agent})
    html = BeautifulSoup(r.text)
    return html.select_one('.percentuale').text.strip()

In [69]:
data = []

circioscrizione = get_panel()
for c_href, c_title in circioscrizione:
    regions = get_panel(c_href)
    for r_href, r_title in regions:
        print(r_title)
        provinces = get_panel(r_href)
        for p_href, p_title in provinces:
            communes = get_panel(p_href)
            for com_href, com_title in communes:
                turnout = get_turnout(com_href)
                data.append((c_title, r_title, p_title, com_title, turnout))

PIEMONTE
VALLE D'AOSTA
LOMBARDIA
LIGURIA
TRENTINO-ALTO ADIGE
VENETO
FRIULI-VENEZIA GIULIA
EMILIA-ROMAGNA
TOSCANA
UMBRIA
MARCHE
LAZIO
ABRUZZO
MOLISE
CAMPANIA
PUGLIA
BASILICATA
CALABRIA
SICILIA
SARDEGNA


In [73]:
df = pd.DataFrame(data, columns=['circo', 'region', 'province', 'commune', 'turnout'])

In [75]:
df.turnout = df.turnout.str.replace(' %', '')
df.turnout = df.turnout.str.replace(',', '.')
df.turnout = df.turnout.astype(float)

In [ ]:
df.commune = df.commune.str.split('/', expand=True)[0]

In [568]:
df.to_csv('../../data/raw/ita.csv', index=False)

## Merge

In [569]:
geo = gpd.read_file('../../data/shapefiles/ita/gadm36_ITA_3.shp')

In [570]:
geo['NAME_1_UP'] = geo.NAME_1.str.upper()
geo['NAME_2_UP'] = geo.NAME_2.str.upper()
geo['NAME_3_UP'] = geo.NAME_3.str.upper()

In [571]:
geo['NAME_1_UP'] = geo.NAME_1_UP.replace('APULIA', 'PUGLIA')
geo['NAME_1_UP'] = geo.NAME_1_UP.replace('SICILY', 'SICILIA')

In [572]:
geo['NAME_2_UP'] = geo.NAME_2_UP.replace('REGGIO DI CALABRIA', 'REGGIO CALABRIA')
geo['NAME_2_UP'] = geo.NAME_2_UP.replace("FORLI' - CESENA", "FORLI'-CESENA")
geo['NAME_2_UP'] = geo.NAME_2_UP.replace('MANTUA', 'MANTOVA')
geo['NAME_2_UP'] = geo.NAME_2_UP.replace('MONZA AND BRIANZA', 'MONZA E DELLA BRIANZA')
geo['NAME_2_UP'] = geo.NAME_2_UP.replace('SYRACUSE', 'SIRACUSA')
geo['NAME_2_UP'] = geo.NAME_2_UP.replace('FLORENCE', 'FIRENZE')
geo['NAME_2_UP'] = geo.NAME_2_UP.replace('MASSA CARRARA', 'MASSA-CARRARA')
geo['NAME_2_UP'] = geo.NAME_2_UP.replace('PADUA', 'PADOVA')

In [573]:
geo = geo.merge(df,
               left_on=['NAME_1_UP', 'NAME_2_UP', 'NAME_3_UP'],
               right_on=['region', 'province', 'commune'], how='left')

In [574]:
geo['NAME_3_UP_fix'] = geo.NAME_3_UP.str.replace("' ", "'")
geo = geo.merge(df,
               left_on=['NAME_1_UP', 'NAME_2_UP', 'NAME_3_UP_fix'],
               right_on=['region', 'province', 'commune'], how='left',
               suffixes=('', '_x'))
geo = geo[[x for x in geo.columns if not x.endswith('_x') or 'turnout' in x]]

geo.turnout = geo.turnout.fillna(geo.turnout_x)
geo = geo.drop(['NAME_3_UP_fix', 'turnout_x'], axis=1)

In [575]:
geo['NAME_3_UP_fix'] = geo.NAME_3_UP.str.replace("À", "A'")
geo['NAME_3_UP_fix'] = geo.NAME_3_UP_fix.str.replace("Ò", "O'")
geo['NAME_3_UP_fix'] = geo.NAME_3_UP_fix.str.replace("Ù", "U'")
geo['NAME_3_UP_fix'] = geo.NAME_3_UP_fix.str.replace("Ì", "I'")
geo['NAME_3_UP_fix'] = geo.NAME_3_UP_fix.str.replace("È", "E'")
geo = geo.merge(df,
               left_on=['NAME_1_UP', 'NAME_2_UP', 'NAME_3_UP_fix'],
               right_on=['region', 'province', 'commune'], how='left',
               suffixes=('', '_x'))
geo = geo[[x for x in geo.columns if not x.endswith('_x') or 'turnout' in x]]

geo.turnout = geo.turnout.fillna(geo.turnout_x)
geo = geo.drop(['NAME_3_UP_fix', 'turnout_x'], axis=1)

In [576]:
geo.NAME_3_UP = geo.NAME_3_UP.replace('NOCERA TIRINESE', 'NOCERA TERINESE')
geo.NAME_3_UP = geo.NAME_3_UP.replace('JONADI', 'IONADI')
geo.NAME_3_UP = geo.NAME_3_UP.replace("SANT' AGATA DE' GOTI", "SANT'AGATA DE' GOTI")
geo.NAME_3_UP = geo.NAME_3_UP.replace('TRENTOLA-DUCENTA', 'TRENTOLA DUCENTA')
geo.NAME_3_UP = geo.NAME_3_UP.replace("CITTÀ SANT' ANGELO", "CITTA' SANT'ANGELO")
geo.NAME_3_UP = geo.NAME_3_UP.replace('CASSANO ALLO IONIO', "CASSANO ALL'IONIO")
geo.NAME_3_UP = geo.NAME_3_UP.replace('JOLANDA DI SAVOIA', 'IOLANDA DI SAVOIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('VERRÈS', 'VERRES')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MONTORO INFERIORE', 'MONTORO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MONTORO SUPERIORE', 'MONTORO')
# Hm
geo.NAME_3_UP = geo.NAME_3_UP.replace('BAZZANO', 'VALSAMOGGIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CASTELLO DI SERRAVALLE', 'VALSAMOGGIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CRESPELLANO', 'VALSAMOGGIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MONTEVEGLIO', 'VALSAMOGGIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('SAVIGNO', 'VALSAMOGGIA')
# /hm
geo.NAME_3_UP = geo.NAME_3_UP.replace('MASSA FISCAGLIA', 'FISCAGLIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MIGLIARINO', 'FISCAGLIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MIGLIARO', 'FISCAGLIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('SISSA', 'SISSA TRECASALI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('TRECASALI', 'SISSA TRECASALI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('POGGIO BERNI', 'POGGIO TORRIANA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('TORRIANA', 'POGGIO TORRIANA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('DUINO-AURISINA', 'DUINO AURISINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CAMPOLONGO AL TORRE', 'CAMPOLONGO TAPOGLIANO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('TAPOGLIANO', 'CAMPOLONGO TAPOGLIANO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('RIVIGNANO', 'RIVIGNANO TEOR')
geo.NAME_3_UP = geo.NAME_3_UP.replace('TEOR', 'RIVIGNANO TEOR')
geo.NAME_3_UP = geo.NAME_3_UP.replace('POZZAGLIA SABINO', 'POZZAGLIA SABINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('AQUILA DI ARROSCIA', 'AQUILA D\'ARROSCIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('SAN REMO', 'SANREMO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('BREMBILLA', 'VAL BREMBILLA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GEROSA', 'VAL BREMBILLA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('COSTA DI SERINA', 'COSTA SERINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace("SANT' OMOBONO IMAGNA", "SANT'OMOBONO TERME")
geo.NAME_3_UP = geo.NAME_3_UP.replace('VALSECCA', "SANT'OMOBONO TERME")
geo.NAME_3_UP = geo.NAME_3_UP.replace('RODENGO-SAIANO', 'RODENGO SAIANO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('TOSCOLANO-MADERNO', 'TOSCOLANO MADERNO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CIVENNA', 'BELLAGIO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CONSIGLIO DI RUMO', 'GRAVEDONA ED UNITI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GRAVEDONA', 'GRAVEDONA ED UNITI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GERMASINO', 'GRAVEDONA ED UNITI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('DREZZO', 'COLVERDE')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GIRONICO', 'COLVERDE')
geo.NAME_3_UP = geo.NAME_3_UP.replace('PARÈ', 'COLVERDE')
geo.NAME_3_UP = geo.NAME_3_UP.replace("VILLA D' ALMÈ", "VILLA D'ALME'")
geo.NAME_3_UP = geo.NAME_3_UP.replace('LENNO', 'TREMEZZINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MEZZEGRA', 'TREMEZZINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('OSSUCCIO', 'TREMEZZINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('TREMEZZO', 'TREMEZZINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace("CA' D' ANDREA", "CA' D'ANDREA")
geo.NAME_3_UP = geo.NAME_3_UP.replace('CASALE CREMASCO-VIDOLASCO', 'CASALE CREMASCO VIDOLASCO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('VERDERIO INFERIORE', 'VERDERIO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('VERDERIO SUPERIORE', 'VERDERIO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('BORGOFORTE', 'BORGO VIRGILIO')
geo.NAME_3_UP = geo.NAME_3_UP.replace('VIRGILIO', 'BORGO VIRGILIO')
geo.NAME_3_UP = geo.NAME_3_UP.replace("BASTIDA DE' DOSSI", 'CORNALE E BASTIDA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CORNALE', 'CORNALE E BASTIDA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GODIASCO', 'GODIASCO SALICE TERME')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GABBIONETA-BINANUOVA', 'GABBIONETA BINANUOVA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GORNATE-OLONA', 'GORNATE OLONA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('LAVENA PONTE TRESA', 'LAVENA-PONTE TRESA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('PINO SULLA SPONDA DEL LAGO MAGGIORE', 'MACCAGNO CON PINO E VEDDASCA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MACCAGNO', 'MACCAGNO CON PINO E VEDDASCA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('VEDDASCA', 'MACCAGNO CON PINO E VEDDASCA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CASTEL COLONNA', 'TRECASTELLI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('RIPE', 'TRECASTELLI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MONTERADO', 'TRECASTELLI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('COLBORDOLO', 'VALLEFOGLIA')
geo.NAME_3_UP = geo.NAME_3_UP.replace("SANT' ANGELO IN LIZZOLA", 'TRECASTELLI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('PUEGNAGO SUL GARDA', 'PUEGNAGO DEL GARDA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('GADESCO-PIEVE DELMONA', 'GADESCO PIEVE DELMONA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('RIVANAZZANO', 'RIVANAZZANO TERME')
geo.NAME_3_UP = geo.NAME_3_UP.replace('MONTE GRIMANO', 'MONTE GRIMANO TERME')
geo.NAME_3_UP = geo.NAME_3_UP.replace("SANT' AGATA FELTRIA", "SANT'AGATA FELTRIA")
geo.NAME_3_UP = geo.NAME_3_UP.replace('LONATO', 'LONATO DEL GARDA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('BRIGNANO-FRASCATA', 'BRIGNANO FRASCATA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('CERRINA MONFERRATO', 'CERRINA')
geo.NAME_3_UP = geo.NAME_3_UP.replace('VIALE', "VIALE D'ASTI")
geo.NAME_3_UP = geo.NAME_3_UP.replace('DONORÌ', 'DONORI')
geo.NAME_3_UP = geo.NAME_3_UP.replace('BELVI', "BELVI'")
geo.NAME_3_UP = geo.NAME_3_UP.replace('GALTELLI', "GALTELLI'")
geo.NAME_3_UP = geo.NAME_3_UP.replace('ONANI', "ONANI'")
geo.NAME_3_UP = geo.NAME_3_UP.replace("TRINITÀ D' AGULTU E VIGNOLA", "TRINITA' D'AGULTU E VIGNOLA")
geo.NAME_3_UP = geo.NAME_3_UP.replace('OLLASTRA SIMÁXIS', 'OLLASTRA')
geo.NAME_3_UP = geo.NAME_3_UP.replace("SAN NICOLÒ D' ARCIDANO", "SAN NICOLO' D'ARCIDANO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("ULÀ TIRSO", "ULA TIRSO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("NUGHEDU DI SAN NICOLÒ", "NUGHEDU SAN NICOLO'")
geo.NAME_3_UP = geo.NAME_3_UP.replace("FORZA D' AGRÒ", "FORZA D'AGRO'")
geo.NAME_3_UP = geo.NAME_3_UP.replace("GIARDINI-NAXOS", "GIARDINI NAXOS")
geo.NAME_3_UP = geo.NAME_3_UP.replace("MAZZARRÀ SANT' ANDREA", "MAZZARRA' SANT'ANDREA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CASTRONUOVO DI SICILIA", "CASTRONOVO DI SICILIA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CALATAFIMI-SEGESTA", "CALATAFIMI SEGESTA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CASTELFRANCO DI SOPRA", "CASTELFRANCO PIANDISCO'")
geo.NAME_3_UP = geo.NAME_3_UP.replace("PIAN DI SCO", "CASTELFRANCO PIANDISCO'")
geo.NAME_3_UP = geo.NAME_3_UP.replace("PRATOVECCHIO", "PRATOVECCHIO STIA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("STIA", "PRATOVECCHIO STIA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("FIGLINE VALDARNO", "FIGLINE E INCISA VALDARNO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("INCISA IN VAL D' ARNO", "FIGLINE E INCISA VALDARNO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("SAN PIERO A SIEVE", "SCARPERIA E SAN PIERO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("SCARPERIA", "SCARPERIA E SAN PIERO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("FABBRICHE DI VALLICO", "FABBRICHE DI VERGEMOLI")
geo.NAME_3_UP = geo.NAME_3_UP.replace("VERGEMOLI", "FABBRICHE DI VERGEMOLI")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CASCIANA TERME", "CASCIANA TERME LARI")
geo.NAME_3_UP = geo.NAME_3_UP.replace("LARI", "CASCIANA TERME LARI")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CRESPINA", "CRESPINA LORENZANA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("LORENZANA", "CRESPINA LORENZANA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("BEZZECCA", "LEDRO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("PIEVE DI LEDRO", "LEDRO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CONCEI", "LEDRO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("MOLINA DI LEDRO", "LEDRO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("TIARNO DI SOPRA", "LEDRO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("TIARNO DI SOTTO", "LEDRO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("BLEGGIO INFERIORE", "COMANO TERME")
geo.NAME_3_UP = geo.NAME_3_UP.replace("LOMASO", "COMANO TERME")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CADERZONE", "CADERZONE TERME")
geo.NAME_3_UP = geo.NAME_3_UP.replace("DRO", "DRO'")
geo.NAME_3_UP = geo.NAME_3_UP.replace("RONCEGNO", "RONCEGNO TERME")
geo.NAME_3_UP = geo.NAME_3_UP.replace("RUFFRÈ", "RUFFRE'-MENDOLA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("EMARÈSE", "EMARESE")
geo.NAME_3_UP = geo.NAME_3_UP.replace("FÉNIS", "FENIS")
geo.NAME_3_UP = geo.NAME_3_UP.replace("JOVENÇAN", "JOVENCAN")
geo.NAME_3_UP = geo.NAME_3_UP.replace("SAINT-RHÉMY-EN-BOSSES", "SAINT-RHEMY-EN-BOSSES")
geo.NAME_3_UP = geo.NAME_3_UP.replace("QUERO", "QUERO VAS")
geo.NAME_3_UP = geo.NAME_3_UP.replace("VAS", "QUERO VAS")
geo.NAME_3_UP = geo.NAME_3_UP.replace("IESOLO", "JESOLO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("SANTO STINO DI LIVENZA", "SAN STINO DI LIVENZA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("BRENZONE", "BRENZONE SUL GARDA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("ALBISOLA MARINA", "ALBISSOLA MARINA")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CERRETO LANGHE", "CERRETTO LANGHE")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CASTELLAVAZZO", "LONGARONE")
geo.NAME_3_UP = geo.NAME_3_UP.replace("CORNEDO ALL' ISARCO", "CORNEDO ALL'ISARCO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("FIÈ ALLO SCILIAR", "FIE' ALLO SCILIAR")
geo.NAME_3_UP = geo.NAME_3_UP.replace("MAGRÈ SULLA STRADA DEL VINO", "MAGRE' SULLA STRADA DEL VINO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("MONGUELFO", "MONGUELFO-TESIDO")
geo.NAME_3_UP = geo.NAME_3_UP.replace("TRODENA", "TRODENA NEL PARCO NATURALE")

In [577]:
geo = geo.merge(df[['region', 'province', 'commune', 'turnout']],
               left_on=['NAME_1_UP', 'NAME_2_UP', 'NAME_3_UP'],
               right_on=['region', 'province', 'commune'],
               how='left',
               suffixes=('', '_x'))
geo = geo[[x for x in geo.columns if not x.endswith('_x') or 'turnout' in x]]

geo.turnout = geo.turnout.fillna(geo.turnout_x)
geo = geo.drop('turnout_x', axis=1)

In [578]:
geo.loc[(geo.NAME_3_UP == 'CORTENOVA') & (geo.NAME_2_UP == 'BERGAMO'), 'NAME_3_UP'] = 'CORTENUOVA'
geo.loc[(geo.NAME_3_UP == 'SAN COLOMBANO AL LAMBRO') & (geo.NAME_2_UP == 'LODI'), 'NAME_2_UP'] = 'MILANO'
geo.loc[(geo.NAME_3_UP == 'RONCELLO') & (geo.NAME_2_UP == 'MILANO'), 'NAME_2_UP'] = 'MONZA E DELLA BRIANZA'
for loc in [4559, 4572, 4587, 4590, 4601, 4603, 4611]:
    geo.loc[loc, 'NAME_1_UP'] = 'EMILIA-ROMAGNA'
    geo.loc[loc, 'NAME_2_UP'] = 'RIMINI'
geo.loc[4604, 'NAME_2_UP'] = 'ANCONA'

In [579]:
geo.loc[6075, 'NAME_2_UP'] = 'OLBIA-TEMPIO'
geo.loc[6079, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6080, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6085, 'NAME_2_UP'] = 'ORISTANO'
geo.loc[6086, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6088, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6089, 'NAME_2_UP'] = 'ORISTANO'
geo.loc[6100, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6101, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6102, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6113, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6120, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6121, 'NAME_2_UP'] = 'OLBIA-TEMPIO'
geo.loc[6123, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6124, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6133, 'NAME_2_UP'] = 'CAGLIARI'
geo.loc[6139, 'NAME_2_UP'] = 'CAGLIARI'

In [580]:
geo = geo.merge(df[['region', 'province', 'commune', 'turnout']],
               left_on=['NAME_1_UP', 'NAME_2_UP', 'NAME_3_UP'],
               right_on=['region', 'province', 'commune'],
               how='left',
               suffixes=('', '_x'))
geo = geo[[x for x in geo.columns if not x.endswith('_x') or 'turnout' in x]]

geo.turnout = geo.turnout.fillna(geo.turnout_x)
geo = geo.drop('turnout_x', axis=1)

## Export

In [584]:
geo = geo[['NAME_3', 'turnout', 'geometry']]
geo = geo.rename(columns={'NAME_3': 'name'})
geo['country'] = 'ita'

In [586]:
geo.to_file('../../data/processed/ita')